# Input

In [1]:
with open("day_7.txt", "r") as f:
    inp = f.readlines()

# Utils

In [2]:
import re

def parse_line(line):
    match = re.match("(\w+ \w+) bags contain ((?:\d \w+ \w+ bags*(?:, )*)*).", line)
    bag, content = match.group(1, 2)
    if content:
        content = [re.match("(\d+) (\w+ \w+) bags*", bags).group(1, 2) for bags in content.split(", ")]
        content_dict = {bag: nb for nb, bag in content}
    else:
        content_dict = {}
        
    return bag, content_dict


In [3]:
import numpy as np

rule_dict = {}
for line in inp:
    name, content = parse_line(line)
    rule_dict[name] = content

# Part 1

In [4]:
# Generation of an directed graph
indexes = list(rule_dict.keys()) # Define an index for each bag
rule_graph = np.zeros((len(rule_dict), len(rule_dict))) # Empty graph matrix
for outer_bag, content in rule_dict.items(): # Iteration on the bag dictionnary
    outer_bag_ind = indexes.index(outer_bag) # Get the index of the bag 
    for inner_bag, number in content.items(): # Iteration on the content of the bag
        inner_bag_ind = indexes.index(inner_bag) # Get the index of the inner bag
        rule_graph[outer_bag_ind][inner_bag_ind] = number # Specify the number of bags as the weight of the link between the inner and outer bag
        
        
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import shortest_path

graph = csr_matrix(rule_graph) # Transformation to a numpy graph
shortest_graph = shortest_path(graph) # Compute the minimal distance from one node to another. As we are in a directed graph, if the shortest path between a node A and a node B is infinite, then the bag B is not include in the bag A

# Now we look, for the given bag, which nodes have a finite distance to this node.
counter = 0
for distance in shortest_graph[:,indexes.index("shiny gold")]:
    if distance != np.inf and distance != 0:
        counter += 1
print(counter)

222


# Part 2

In [5]:
# Simple recursive function : for each inner bag we add the number of this bag and then run the function for the inner bag...
def get_number_of_bags(name):
    # Termination condition : if the inner bag is empty, we return 0
    if rule_dict[name] == {}:
        return 0
    
    counter = 0
    for bags, count in rule_dict[name].items():
        counter += int(count) + int(count)*get_number_of_bags(bags)
        
    return counter

In [6]:
get_number_of_bags("shiny gold")

13264